<a href="https://colab.research.google.com/github/swguo/Generating-Personalized-Phishing-Emails/blob/main/GPT2_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%time
%%capture
!pip install transformers==4.5.0

CPU times: user 40.5 ms, sys: 13.2 ms, total: 53.8 ms
Wall time: 5.13 s


In [7]:
!nvidia-smi

Wed Mar 16 08:35:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, \
                         Trainer, TrainingArguments

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler
import pprint
pp = pprint.PrettyPrinter(indent=4)
from IPython.display import clear_output

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.10.0+cu111


# Configurations

In [9]:
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2-medium' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 128

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Loading Tokenizer, Config and Model

In [11]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print(len(tokenizer))
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                          bos_token_id=tokenizer.bos_token_id,
                          eos_token_id=tokenizer.eos_token_id,
                          sep_token_id=tokenizer.sep_token_id,
                          pad_token_id=tokenizer.pad_token_id,
                          output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                          pad_token_id=tokenizer.eos_token_id,
                          output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [12]:
def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

# Download Spear Email model - GPT2


In [3]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=c6cd10f6f5f64eafbd5a87eb355cf1f651e2a8718ea8c419204be2de55131b82
  Stored in directory: /tmp/pip-ephem-wheel-cache-7m_ijmms/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [4]:
!gdown --id 1dPhbShbTwBB_3cdsV4zp0gLFD0mnREIO

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dPhbShbTwBB_3cdsV4zp0gLFD0mnREIO
To: /content/pytorch_model.bin
100% 1.44G/1.44G [00:06<00:00, 230MB/s]


In [13]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)

model = get_model(tokenizer, 
          special_tokens=SPECIAL_TOKENS,
          load_model_path='pytorch_model.bin')

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

50262
Special tokens added


Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

# Generation

## Introduction to input fields

Title => Email Subject 

Type => has two choices
 * Fraud  : Fraud type
 * Normal  : BEC type

Category => Topic
 * BUSINESS
 * MONEY

Formats => Body formats 
 * News
 * Email

Keywords => Expected keyowrd (list type)

## Define Function

In [14]:
import re
def remove_Symbol(s):
    s = re.sub(r'[^\w]','',s)
    return s


In [15]:
def generation_token(title,text,keywords,types,category,formats):    
    
    kw = join_keywords(keywords, randomize=False)
    prompt = SPECIAL_TOKENS['bos_token'] + types + \
             SPECIAL_TOKENS['sep_token'] + category + \
             SPECIAL_TOKENS['sep_token'] + formats + \
             SPECIAL_TOKENS['sep_token'] + title + \
             SPECIAL_TOKENS['sep_token'] + kw + \
             SPECIAL_TOKENS['sep_token']
    
    prompt_len = len(types)+len(category)+len(formats)+ len(title) + len(','.join(keywords)) 
    
    generated = tokenizer([prompt],max_length=MAXLEN, return_tensors='pt')['input_ids'].to(device)
    

    model.eval()

    sample_outputs = model.generate(
        generated,
        do_sample=False, 
        max_length=MAXLEN, 
        top_k=1,
        top_p=0.75,
        num_return_sequences=3
    )
    
    return sample_outputs,len(prompt)

## 1. Generation phishing emails for fraud

## Topic for COIVD19 of Fraud Emails

In [74]:
title = "please reply to me soon "
#title = "please complete as soon as possible "
#title = 'I will get back to you in the am '
#title = 'We are keenly interested in setting up a new hospital '
#title = 'How about was that payment? reply to me soon '
#title = "I will get back to you in the am "

keywords = ['covid19', 'case', 'hospital','CDC']

In [75]:
types = "Fraud"
category = "BUSINESS"
formats = "Email"
kw = join_keywords(keywords, randomize=False)
prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [94]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
the diagnosis was made in an urgent and very confidential manner by drdrdavidgomezcavastisymodianadoctor please replyto me soonmy name is dr david 

gombaa native of cotonounigeriamy parents were murderedfor political reasons duringthe regimeof mr david sekoto who ruled from 1983 until he 

died on this 21st april 2000 after beingdiagnosedand then transferred into detention at our countrysembassy herein nigeria for medical treatment 

my father had already gone through several doctors before finally getting hislast breath when we found out thathe succumbed along 

side many other people including myself and three 

Phishing Email : 2
i was told that you were in icu with covid-19 and had diedi have not been able for the past 2 

weeks or so because of this illness but am very hopeful that things will be all right please lets discuss 

next timethank youdrdavid 

Phishing Email : 3
i will be in touch with you asapi have been diagnosedwith covasysmogulism and am undergo

## Topic for Finace of Fraud Emails

In [69]:
title = 'How was that payment? reply to me soon '
#title = 'I will get back to you in the am '
#title = 'please, reply to me soon '
keywords = ['bank','account','number','reply','credit']

In [70]:
types = "Fraud" 
category = "BUSINESS"
formats = "Email"
kw = join_keywords(keywords, randomize=False)


prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [73]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
i am mrdaniel smith from south africamy name is daniel jnrbankauvergneafricawhy did those funds arrive atyour bank during this time periodreply 

mine now so we can discuss further about how bestto transfer these money outof our accounts for investment purposesthankyou very 

muchbest regardsmrdavidsmith 

Phishing Email : 2
i will be very grateful if you can give my a few days noticei am mr michael kowalski of the bank 

of africa boa here in dakar senegal and have an urgentand profitablebusiness proposition foryouthis is necessitated bythe natureof this business 

proposalbefore we proceed however please considerthat there aretwo ways forwardfirstly it may come as surprise but i believe everyone knows 

how importantit has been paying off credit cards since its inception hence whyi decidednot contact any bank or financial institution 

before makinga deposit with them secondly some banksfinancial institutions might feel threatened because theyare not considered for

## 2. Generation phishing emails for BEC

## Topic for COIVD19 of BEC Emails

In [37]:
title = "please reply to me soon "
#title = "please complete as soon as possible "
#title = 'I will get back to you in the am '
#title = 'We are keenly interested in setting up a new hospital '
#title = 'How about was that payment? reply to me soon '
#title = "I will get back to you in the am "
types = "Normal" 
category = "BUSINESS"
formats = "Email"
keywords = ['covid19', 'case', 'hospital','CDC']
kw = join_keywords(keywords, randomize=False)


prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [50]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
i am in the hospital bed with a cot on top of us that is being kept from oxygen and heat 

so we are not able to move around or breathei have been diagnosedwith covid-19 which means theres no cell phone 

receptioncavids please reply via email shortly we will be transferred into icu where they cant communicate wuscvad19cellddccontact you sooner 

Phishing Email : 2
i have not seen him in over a year so this is the first time that he has spoken publicly about 

his condition since being hospitalized last week with covid-19 at columbia university 

Phishing Email : 3
i have not seen you before but hope that we will meet again in person or by phone for a more 

informal discussion about the situation at our hospital and how best it can be resolvedi am drdavid covidine an assistant 

professor of medicine with columbia university schoolof public health the director is also chief medical officerfor cdcnyu and serves as 

deputydirector of operationsfor over 20 years he ha

## Topic for finance of BEC Emails

In [51]:
title = 'How was that payment? reply to me soon '
#title = 'I will get back to you in the am '
#title = 'please, reply to me soon '
keywords = ['bank','account','number','reply','credit']
#keywords = ['bank','account','reply','number','credit']

In [52]:
types = "Normal"
category = "BUSINESS"
formats = "Email"

kw = join_keywords(keywords, randomize=False)


prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
print(prompt)
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)
print(generated)
model.eval();

<|BOS|>Normal<|SEP|>BUSINESS<|SEP|>Email<|SEP|>How was that payment? reply to me soon <|SEP|>bank,account,number,reply,credit
tensor([[50257, 26447, 50261, 45346, 44180, 50261, 15333, 50261,  2437,   373,
           326,  6074,    30, 10971,   284,   502,  2582,   220, 50261, 17796,
            11, 23317,    11, 17618,    11, 47768,    11, 43082]],
       device='cuda:0')


In [68]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
the account is in my name and has been dormant since we were married we have two childreni am contacting you 

because of this business opportunity which will be very beneficialto both parties please acceptmy sincere apologizes if it appears as 

thoughyou are being treated unfairly or otherwisedisparately bythis companybankaccountsbankhowwas that payments amount reply to me soon 

Phishing Email : 2
reply back in a few weeksi will be very grateful if we can do this together 

Phishing Email : 3
5m in deposits and withdrawals from your bank account with a balance of us78 million three hundred thousand dollars for twelve 

calendar months answer the number is simply an estimation based on my personal knowledge about how much money i have 

available at any given time 

